In [14]:
from datetime import date, timedelta
import win32com.client
from win32com.client import Dispatch, constants
from openpyxl import load_workbook
from office365.runtime.auth.user_credential import UserCredential
from office365.sharepoint.client_context import ClientContext
from office365.runtime.auth.client_credential import ClientCredential
from office365.sharepoint.client_context import ClientContext
import json

In [15]:
f = open(r'T:\DCProjects\GitHub\LCOG_support\config\keys.json')
crd = json.load(f)
f.close

<function TextIOWrapper.close()>

In [16]:
def get_sharepoint_context_using_user():
 
    # Get sharepoint credentials
    sharepoint_url = 'https://lanecouncilofgovernments.sharepoint.com'

    # Initialize the client credentials
    user_credentials = UserCredential(crd['SharePoint']['usr'], crd['SharePoint']['pwd'])

    # create client context object
    ctx = ClientContext(sharepoint_url).with_credentials(user_credentials)

    return ctx

In [17]:
def get_files(file_url):
    try:

        site_url = "https://lanecouncilofgovernments.sharepoint.com"
        credentials = ClientCredential(crd['SharePoint']['usr'], crd['SharePoint']['pwd'])

        ctx = ClientContext(site_url).with_credentials(credentials)
        
        # file_url is the sharepoint url from which you need the list of files
        list_source = ctx.web.get_folder_by_server_relative_url(file_url)
        files = list_source.files
        ctx.load(files)
        ctx.execute_query()

        return files

    except Exception as e:
        print(e)

In [18]:
get_files('https://lanecouncilofgovernments.sharepoint.com/:x:/r/sites/LCOGDiversityEquityBelonging/_layouts/15/Doc.aspx?sourcedoc=%7B95B76D95-363F-452E-A99D-77524AA7A68E%7D&file=Facilitator%20and%20Notetaker%20Rotation.xlsx&action=default&mobileredirect=true')

{"error":"invalid_client","error_description":"AADSTS70002: Client application name 'dchen@lcog.org@9a80ddb7-1790-4782-a634-ef32f273169c' is not valid or the credentials used to authenticate the client could not be understood by the server.\r\nTrace ID: 823afaab-9e6c-4a0a-a3cf-df4c10170000\r\nCorrelation ID: 8e920b93-2e3f-4510-a3f4-2f7c09632be8\r\nTimestamp: 2023-02-09 19:58:56Z","error_codes":[70002],"timestamp":"2023-02-09 19:58:56Z","trace_id":"823afaab-9e6c-4a0a-a3cf-df4c10170000","correlation_id":"8e920b93-2e3f-4510-a3f4-2f7c09632be8","error_uri":"https://accounts.accesscontrol.windows.net/error?code=70002"}


In [11]:
import datetime

def third_Wednesday(year, month):
    """Return datetime.date for monthly option expiration given year and
    month
    """
    # The 15th is the lowest third day in the month
    third = datetime.date(year, month, 15)
    # What day of the week is the 15th?
    w = third.weekday()
    # Wednesday is weekday 2
    if w != 2:
        # Replace just the day (of month)
        third = third.replace(day=(15 + (2 - w) % 7))
    return third

In [12]:
date1 = third_Wednesday(2023, 2)

In [13]:
date1

datetime.date(2023, 2, 15)

In [6]:
date2 = date.today() + timedelta(days=7)

In [7]:
const=win32com.client.constants
olMailItem = 0x0
obj = win32com.client.Dispatch("Outlook.Application")
newMail = obj.CreateItem(olMailItem)
newMail.Subject = f'Reminder: DEIB meeting next Wednesday {str(date1)}'
newMail.BodyFormat = 2 # olFormatHTML https://msdn.microsoft.com/en-us/library/office/aa219371(v=office.11).aspx
newMail.HTMLBody =  """
<HTML><BODY><p>Hello all,</p> 

<p>Our meeting facilitator and notetaker next week are Denise and Kate, and next month are Ellen and Heidi. Following the links, please find the folders for the <a href="https://lanecouncilofgovernments.sharepoint.com/:f:/r/sites/LCOGDiversityEquityBelonging/Shared%20Documents/Committee%20Meetings/Agendas/2023?csf=1&web=1&e=wBl76p">meeting agenda</a>, and <a href="https://lanecouncilofgovernments.sharepoint.com/:f:/r/sites/LCOGDiversityEquityBelonging/Shared%20Documents/Committee%20Meetings/Meeting%20Notes/2023?csf=1&web=1&e=03DAbf">meeting notes</a>.

Meeting facilitators will need to fill out the agenda details. Please note that we will have a hybrid meeting this time, and the in-person meeting room is McKenzie Room on PPB 4th floor.</p> 

<p>Have a great day!</p>

<p>Dongmei on behalf of DEIB Strategic Planning Subcommittee</p></BODY></HTML>
"""
newMail.To = "LCOGDiversityEquityInclusionBelonging@lcog.org"
#newMail.From = "LCOGDiversityEquityInclusionBelonging@lcog.org"

newMail.Send()